<a href="https://colab.research.google.com/github/vionakaleb/colab-python/blob/main/LaptopPriceKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

# Load the dataset
data = pd.read_csv('laptopData.csv')

# Display first rows
print(data.head())

# Replace '?' with NaN
data.replace('?', np.nan, inplace=True)

# Convert to float
data['Ram'] = data['Ram'].str.extract('(\d+)').astype(float)
data['Cpu'] = data['Cpu'].str.extract('([\d.]+)GHz').astype(float)
data['Memory'] = data['Memory'].str.extract('(\d+)').astype(float)
data['Inches'] = data['Inches'].str.extract('([\d.]+)"').astype(float)

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
# Select the columns for imputation
columns_to_impute = ['Ram', 'Cpu', 'Memory', 'Inches']
# Impute the selected columns
data[columns_to_impute] = imputer.fit_transform(data[columns_to_impute])


# Drop rows with missing values
data.dropna(inplace=True)

# Select target variable
X = data[['Ram', 'Cpu', 'Memory', 'Inches']]
y = data['Price']

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the sequential model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile the model with SGD optimizer and mean squared error loss
model.compile(optimizer=SGD(lr=0.01), loss='mean_squared_error')

# Train the model with a specified number of epochs
epochs = 50  # Adjust the number of epochs as needed
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Visualize the model
plt.scatter(y_test, y_pred)
plt.xlabel('True Prices')
plt.ylabel('Predicted Prices')
plt.title('True Prices vs. Predicted Prices')
plt.show()

# Predict the laptop price for new data
new_laptop_data = np.array([[8, 3.0, 512, 15.6]])  # New laptop data
new_laptop_data = scaler.transform(new_laptop_data)  # Standardize the data
predicted_price = model.predict(new_laptop_data)
print('Predicted Price:', predicted_price[0][0])


   Unnamed: 0 Company   TypeName Inches                    ScreenResolution  \
0         0.0   Apple  Ultrabook   13.3  IPS Panel Retina Display 2560x1600   
1         1.0   Apple  Ultrabook   13.3                            1440x900   
2         2.0      HP   Notebook   15.6                   Full HD 1920x1080   
3         3.0   Apple  Ultrabook   15.4  IPS Panel Retina Display 2880x1800   
4         4.0   Apple  Ultrabook   13.3  IPS Panel Retina Display 2560x1600   

                          Cpu   Ram               Memory  \
0        Intel Core i5 2.3GHz   8GB            128GB SSD   
1        Intel Core i5 1.8GHz   8GB  128GB Flash Storage   
2  Intel Core i5 7200U 2.5GHz   8GB            256GB SSD   
3        Intel Core i7 2.7GHz  16GB            512GB SSD   
4        Intel Core i5 3.1GHz   8GB            256GB SSD   

                            Gpu  OpSys  Weight        Price  
0  Intel Iris Plus Graphics 640  macOS  1.37kg   71378.6832  
1        Intel HD Graphics 6000  macOS  

ValueError: ignored